In [1]:
import pandas as pd

In [2]:
import psycopg2
import socket

# information used to create a database connection
sqluser = 'cades' if socket.gethostname() == 'osl-va' else 'sudarshan'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser, host="/var/run/postgresql")

In [3]:
cats = pd.read_csv('cats.csv')
max_limit = 10

queries = []
for category, n_notes in zip(cats['category'], cats['number_of_notes']):
    limit = min(max_limit, n_notes)
    if limit == max_limit:
        q = f"""
        select * from correctnotes where category=\'{category}\' order by random() limit {limit};
        """
    else:
        q = f"""
        select * from correctnotes where category=\'{category}\';
        """
    queries.append(q)

dfs = []
for q in queries:
    df = pd.read_sql_query(q, con)
    dfs.append(df)
    
df = pd.concat(dfs)
df.set_index('row_id', inplace=True)
df.shape

(140, 9)